<a href="https://colab.research.google.com/github/tallerzalan/Deep-Learning-Summer-School/blob/main/DS833_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day 1. (Monday 8th August):

## Imports

In [ ]:
%matplotlib inline

In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf
tf.random.set_seed(42)
import keras
from tensorflow.keras.models import Sequential, Model, load_model
from keras.layers import Input, Conv2D, MaxPooling2D, Activation, Dense, Dropout, Flatten, BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.datasets import mnist
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.optimizers import SGD
from keras.preprocessing.image import load_img, img_to_array
from keras import backend as K

# Scikit-Learn
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import confusion_matrix, classification_report

# Commonly used modules
import statsmodels.api as sm
import numpy as np
from numpy import mean, std, argmax
np.random.seed(24)

# Images, plots, display, and visualization
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

print(tf.__version__)

## Exercise 0:

### NumPy

In [ ]:
a = np.full((2, 3), 4)
b = np.array([[1, 2, 3], [4, 5, 6]])
c = np.eye(2, 3)
d = a + b + c

print(d)

In [ ]:
a = np.array([[1, 2, 3, 4, 5],
              [5, 4, 3, 2, 1],
              [6, 7, 8, 9, 0],
              [0, 9, 8, 7, 6]])

row_sum = np.sum(a, axis = 0)
col_sum = np.sum(a, axis = 1)
transpose = np.transpose(a)

print(row_sum)
print(col_sum)
print(transpose)

### Pandas

In [ ]:
url = 'https://github.com/selva86/datasets/blob/master/Auto.csv?raw=true'
df = pd.read_csv(url)

print(df.head())

In [ ]:
df_1 = df[df['mpg'] > 16]

print(df_1.head())

In [ ]:
df_2 = df[['weight', 'acceleration']].iloc[0:7]

print(df_2)

In [ ]:
df_3 = df[df['horsepower'] != '?']
df_3['horsepower'] = df_3['horsepower'].astype('int')

print(df_3)

In [ ]:
df.describe()

### Matplotlib

In [ ]:
a = np.array([1, 1, 2, 3, 5, 8, 13, 21, 34])
b = np.array([1, 8, 28, 56, 70, 56, 28, 8, 1])

In [ ]:
plt.plot(a, label = 'Training')
plt.plot(b, label = 'Validation')

plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc = 'upper right')
plt.title('Training and Validation Accuracy')

plt.show()

# Day 2. (Tuesday 9th August):

## Exercise 1:

### Data Cleaning

In [ ]:
# The MNIST data is split between 60,000 28 x 28 pixel training images and 10,000 28 x 28 pixel images
(train_data, train_labels), (test_data, test_labels) = mnist.load_data()

print("X_train shape", train_data.shape)
print("y_train amount", train_labels.shape)
print("X_test shape", test_data.shape)
print("y_test amount", test_labels.shape)

In [ ]:
train_data = train_data.reshape(60000, 28*28) # reshape 60,000 28 x 28 matrices into 60,000 784-length vectors.
test_data = test_data.reshape(10000, 28*28) # reshape 10,000 28 x 28 matrices into 10,000 784-length vectors.

train_data = train_data.astype('float32') # change integers to 32-bit floating point numbers
test_data = test_data.astype('float32')

train_data /= 255 # normalize each value for each pixel for the entire vector for each input
test_data /= 255

print("Training matrix shape", train_data.shape)
print("Testing matrix shape", test_data.shape)

In [ ]:
nb_classes = 10 # number of unique digits

train_labels = to_categorical(train_labels, nb_classes)
test_labels = to_categorical(test_labels, nb_classes)

print("Training labels shape", train_labels.shape)
print("Testing labels shape", test_labels.shape)

### Model Fitting

In [ ]:
# The Sequential model is a linear stack of layers and is very common.
model = Sequential()
model.add(Dense(784, activation = 'relu', input_shape = (784,))) # (784,) represents a 784 length vector!
model.add(Dense(256, activation = 'relu'))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(10, activation = 'softmax'))

# Dropout zeroes a selection of random outputs (i.e. disables their activation).
# Dropout helps protect the model from "memorizing" or overfitting the training data.
#model.add(Dropout(0.2))

# The "softmax" activation represents a probability distribution over K different possible outcomes.
# Its values are all non-negative and sum to 1.

# Summarize the built model
model.summary()

In [ ]:
plot_model(model, show_shapes = True)

In [ ]:
# Let's use the Adam optimizer for learning
model.compile(optimizer = 'adam',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

history = model.fit(x = train_data,
                    y = train_labels,
                    epochs = 10,
                    batch_size = 2500,
                    validation_data = (test_data, test_labels))

### Model Evaluation

In [ ]:
history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1, len(loss_values) + 1)

# 'bo' is for blue dot, 'b' is for solid blue line
plt.plot(epochs, loss_values, 'bo', label = 'Training Loss')
plt.plot(epochs, val_loss_values, 'b', label = 'Validation Loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
model.save('MNIST_model.h5')

first_model = load_model('MNIST_model.h5')
first_model.summary()

# Day 3. (Wednesday 10th August):

## Exercise 2:

### EDA

In [ ]:
url = 'https://github.com/selva86/datasets/blob/master/Auto.csv?raw=true'
cars = pd.read_csv(url)
cars = cars[cars.horsepower != '?']

print(df.info())

In [ ]:
cars.describe()

In [ ]:
cars.hist(figsize = (15, 14), bins = 20)
plt.show()

In [ ]:
regplot_cols = cars.select_dtypes(include = np.number).columns.drop('mpg')

for col in regplot_cols:
  sns.regplot(x = col, y = 'mpg', data = cars, line_kws = {"color": "red"})
  plt.show()

In [ ]:
import warnings
warnings.filterwarnings('ignore')

sns.pairplot(cars, diag_kind = 'kde');
#plt.savefig('pairplot.png')

### Regression

#### One Feature

There is a quirk with the L.R. model implementation in statsmodel, therefore we need to explicitly add a constant term to get the intercept.


In [ ]:
cars = sm.add_constant(cars)
model_1var = sm.OLS(cars['mpg'], cars[['const', 'horsepower']])
res_1var = model_1var.fit()
res_1var.summary()

In [ ]:
res_1var.params

#### All Features

In [ ]:
Y = cars['mpg']
X = cars.drop(['const', 'mpg', 'name', 'origin'], axis = 1)
#X = pd.get_dummies(X, columns = ["origin"])
X.head()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

In [ ]:
import statsmodels.formula.api as smf

df_train = pd.concat([y_train, X_train], axis = 1)
reg = smf.ols('mpg ~ X_train', data = df_train)

res = reg.fit()
print(res.summary())

#### Feature Selection

In [ ]:
cols = list(X_train.columns)
pmax = 1
while (len(cols) > 0):
    p= []
    X_1 = X_train[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(y_train,X_1).fit()
    p = pd.Series(model.pvalues.values[1:], index = cols)      
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax > 0.05):
        cols.remove(feature_with_p_max)
    else:
        break

selected_features = cols
print(selected_features)

In [ ]:
newX_train = X_train[selected_features]
reg2 = smf.ols('mpg ~ newX_train', data = df_train)

res2 = reg2.fit()
print(res2.summary())

In [ ]:
from sklearn.linear_model import LinearRegression
regression_model = LinearRegression()

X_train2 = X_train[selected_features]
regression_model.fit(X_train2, y_train)
in_sampleScore = regression_model.score(X_train2, y_train)

X_test2 = X_test[selected_features]
out_sampleScore = regression_model.score(X_test2, y_test)

print(in_sampleScore)
print(out_sampleScore)

In [ ]:
import statistics
ols_model_accuracies = []

for repetition in range(1000):
    (training_inputs,
     testing_inputs,
     training_values,
     testing_values) = train_test_split(X, Y, test_size = 0.2)
    cols = list(X_train.columns)
    pmax = 1
    while (len(cols)>0):
        p= []
        X_1 = training_inputs[cols]
        X_1 = sm.add_constant(X_1)
        ols_model = sm.OLS(training_values, X_1).fit()
        p = pd.Series(ols_model.pvalues.values[1:], index = cols)      
        pmax = max(p)
        feature_with_p_max = p.idxmax()
        if(pmax > 0.05):
            cols.remove(feature_with_p_max)
        else:
            break
    selected_features= cols

    training_inputs = training_inputs[selected_features]
    testing_inputs = testing_inputs[selected_features]
    regression_ols_model = LinearRegression()
    regression_ols_model.fit(training_inputs, training_values)
    classifier_accuracy = regression_ols_model.score(testing_inputs, testing_values)
    ols_model_accuracies.append(classifier_accuracy)
    
print("Minimum Model Accuracy:", min(ols_model_accuracies) * 100)
print("Maximum model Accuracy:", max(ols_model_accuracies) * 100)
print("Median Model Accuracy:", statistics.median(ols_model_accuracies) * 100)

### Regression Improvement

##### $Log10(X)$

In [ ]:
Y = cars['mpg']
X1 = cars.drop(['mpg', 'name', 'origin'], axis = 1)
X1 = np.log10(X1)
X1.head()

X1_train, X1_test, y_train, y_test = train_test_split(X1, Y, test_size = 0.2, random_state = 42)

df_train = pd.concat([y_train, X1_train], axis = 1)
reg1 = smf.ols('mpg ~ X1_train', data = df_train)

res1 = reg1.fit()
print(res1.summary())

##### $\sqrt{X}$

In [ ]:
Y = cars['mpg']
X2 = cars.drop(['mpg', 'name', 'origin'], axis = 1)
X2 = np.sqrt(X2)
X2.head()

X2_train, X2_test, y_train, y_test = train_test_split(X2, Y, test_size = 0.2, random_state = 42)

df_train = pd.concat([y_train, X2_train], axis = 1)
reg1 = smf.ols('mpg ~ X2_train', data = df_train)

res1 = reg1.fit()
print(res1.summary())

##### $1/X$

In [ ]:
Y = cars['mpg']
X3 = cars.drop(['mpg', 'name', 'origin'], axis = 1)
X3 = 1/X3
X3.head()

X3_train, X3_test, y_train, y_test = train_test_split(X3, Y, test_size = 0.2, random_state = 42)

df_train = pd.concat([y_train, X3_train], axis = 1)
reg1 = smf.ols('mpg ~ X3_train', data = df_train)

res1 = reg1.fit()
print(res1.summary())

##### $X^2$

In [ ]:
Y = cars['mpg']
X4 = cars.drop(['mpg', 'name', 'origin'], axis = 1)
X4 = np.square(X4)
X4.head()

X4_train, X4_test, y_train, y_test = train_test_split(X4, Y, test_size = 0.2, random_state = 42)

df_train = pd.concat([y_train, X4_train], axis = 1)
reg1 = smf.ols('mpg ~ X4_train', data = df_train)

res1 = reg1.fit()
print(res1.summary())

# Day 4. (Thursday 11th August):

## Exercise 3:

### Initial Model

In [ ]:
tf.random.set_seed(42)

initial_model = Sequential()
initial_model.add(Dense(784, activation = 'relu', input_shape = (784,)))
initial_model.add(Dense(512, activation = 'relu'))
initial_model.add(Dense(256, activation = 'relu'))
initial_model.add(Dense(128, activation = 'relu'))
initial_model.add(Dense(64, activation = 'relu'))
initial_model.add(Dense(32, activation = 'relu'))
initial_model.add(Dense(16, activation = 'relu'))
initial_model.add(Dense(10, activation = 'softmax'))

initial_model.compile(optimizer = 'adam',
                      loss = 'categorical_crossentropy',
                      metrics = ['accuracy'])

initial_history = initial_model.fit(x = train_data,
                                    y = train_labels,
                                    epochs = 10,
                                    batch_size = 2500,
                                    validation_data = (test_data, test_labels))

initial_history_dict = initial_history.history
loss_values = initial_history_dict['loss']
val_loss_values = initial_history_dict['val_loss']
epochs = range(1, len(loss_values) + 1)

# 'bo' is for blue dot, 'b' is for solid blue line
plt.plot(epochs, loss_values, 'bo', label = 'Training Loss')
plt.plot(epochs, val_loss_values, 'r', label = 'Validation Loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

### L2 Regularized Model Fitting

No additional layer is added if l1 or l2 regularization is used. The regularization is imposed in the Dense layer internally.

In [ ]:
from keras.regularizers import l2
tf.random.set_seed(42)

l2_regularized_model = Sequential()
l2_regularized_model.add(Dense(784, activation = 'relu', kernel_regularizer = l2(0.001), input_shape = (784,)))
l2_regularized_model.add(Dense(512, activation = 'relu', kernel_regularizer = l2(0.001)))
l2_regularized_model.add(Dense(256, activation = 'relu', kernel_regularizer = l2(0.001)))
l2_regularized_model.add(Dense(128, activation = 'relu', kernel_regularizer = l2(0.001)))
l2_regularized_model.add(Dense(64, activation = 'relu', kernel_regularizer = l2(0.001)))
l2_regularized_model.add(Dense(32, activation = 'relu', kernel_regularizer = l2(0.001)))
l2_regularized_model.add(Dense(16, activation = 'relu', kernel_regularizer = l2(0.001)))
l2_regularized_model.add(Dense(10, activation = 'softmax'))

l2_regularized_model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

l2_regularized_history = l2_regularized_model.fit(x = train_data,
                                            y = train_labels,
                                            epochs = 10,
                                            batch_size = 2500,
                                            validation_data = (test_data, test_labels))

l2_regularized_history_dict = l2_regularized_history.history
loss_values = l2_regularized_history_dict['loss']
val_loss_values = l2_regularized_history_dict['val_loss']
epochs = range(1, len(loss_values) + 1)

# 'bo' is for blue dot, 'b' is for solid blue line
plt.plot(epochs, loss_values, 'bo', label = 'Training Loss')
plt.plot(epochs, val_loss_values, 'r', label = 'Validation Loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

### Droupout Layer(s)

Given a dropout rate of 0.45, the Dropout layer randomly removes the fraction of units from participating in the next layer.

E.g. if the first layer has 748 units, after dropout = 0.45 is applied, only (1 - 0.45) * 748 units = 411 units from layer 1 participate in layer 2.

In [ ]:
tf.random.set_seed(42)

dropout_model = Sequential()
dropout_model.add(Dense(784, activation = 'relu', input_shape = (784,)))
dropout_model.add(Dropout(0.45))
dropout_model.add(Dense(512, activation = 'relu'))
dropout_model.add(Dense(256, activation = 'relu'))
dropout_model.add(Dropout(0.45))
dropout_model.add(Dense(128, activation = 'relu'))
dropout_model.add(Dense(64, activation = 'relu'))
dropout_model.add(Dropout(0.45))
dropout_model.add(Dense(32, activation = 'relu'))
dropout_model.add(Dense(16, activation = 'relu'))
dropout_model.add(Dropout(0.45))
dropout_model.add(Dense(10, activation = 'softmax'))

dropout_model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

dropout_history = dropout_model.fit(x = train_data,
                                    y = train_labels,
                                    epochs = 10,
                                    batch_size = 2500,
                                    validation_data = (test_data, test_labels))

dropout_history_dict = dropout_history.history
loss_values = dropout_history_dict['loss']
val_loss_values = dropout_history_dict['val_loss']
epochs = range(1, len(loss_values) + 1)

# 'bo' is for blue dot, 'b' is for solid blue line
plt.plot(epochs, loss_values, 'bo', label = 'Training Loss')
plt.plot(epochs, val_loss_values, 'r', label = 'Validation Loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

### L1 vs. L2

In [ ]:
from keras.regularizers import l1
tf.random.set_seed(42)

l1_regularized_model = Sequential()
l1_regularized_model.add(Dense(784, activation = 'relu', kernel_regularizer = l1(0.0001), input_shape = (784,)))
l1_regularized_model.add(Dense(512, activation = 'relu', kernel_regularizer = l1(0.0001)))
l1_regularized_model.add(Dense(256, activation = 'relu', kernel_regularizer = l1(0.0001)))
l1_regularized_model.add(Dense(128, activation = 'relu', kernel_regularizer = l1(0.0001)))
l1_regularized_model.add(Dense(64, activation = 'relu', kernel_regularizer = l1(0.0001)))
l1_regularized_model.add(Dense(32, activation = 'relu', kernel_regularizer = l1(0.0001)))
l1_regularized_model.add(Dense(16, activation = 'relu', kernel_regularizer = l1(0.0001)))
l1_regularized_model.add(Dense(10, activation = 'softmax'))

l1_regularized_model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

l1_regularized_history = l1_regularized_model.fit(x = train_data,
                                            y = train_labels,
                                            epochs = 10,
                                            batch_size = 5000,
                                            validation_data = (test_data, test_labels))

l1_regularized_history_dict = l1_regularized_history.history
loss_values = l1_regularized_history_dict['loss']
val_loss_values = l1_regularized_history_dict['val_loss']
epochs = range(1, len(loss_values) + 1)

# 'bo' is for blue dot, 'b' is for solid blue line
plt.plot(epochs, loss_values, 'bo', label = 'Training Loss')
plt.plot(epochs, val_loss_values, 'r', label = 'Validation Loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

### Best Model

In [ ]:
tf.random.set_seed(42)

best_model = Sequential()
best_model.add(Dense(784, activation = 'relu', input_shape = (784,)))
best_model.add(Dense(256, activation = 'relu'))
best_model.add(Dense(128, activation = 'relu'))
best_model.add(Dense(10, activation = 'softmax'))

best_model.compile(optimizer = 'adam',
                   loss = 'categorical_crossentropy',
                   metrics = ['accuracy'])

best_history = best_model.fit(x = train_data,
                              y = train_labels,
                              epochs = 10,
                              batch_size = 2500,
                              validation_data = (test_data, test_labels))

best_history_dict = best_history.history
loss_values = best_history_dict['loss']
val_loss_values = best_history_dict['val_loss']
epochs = range(1, len(loss_values) + 1)

# 'bo' is for blue dot, 'b' is for solid blue line
plt.plot(epochs, loss_values, 'bo', label = 'Training Loss')
plt.plot(epochs, val_loss_values, 'r', label = 'Validation Loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Day 6. (Monday 15th August):

## Exercise 4:

### Data Pre-Processing

In [ ]:
(train_X, train_Y), (test_X, test_Y) = mnist.load_data()

print('Training data shape: ', train_X.shape, train_Y.shape)
print('Testing data shape: ', test_X.shape, test_Y.shape)

In [ ]:
classes = np.unique(train_Y)
nClasses = len(classes)

print('Total number of outputs: ', nClasses)
print('Output classes: ', classes)

In [ ]:
plt.figure(figsize = [5, 5])

# Display the first image in training data
plt.subplot(121)
plt.imshow(train_X[0, :, :], cmap = 'gray')
plt.title("Ground Truth: {}".format(train_Y[0]))

# Display the first image in testing data
plt.subplot(122)
plt.imshow(test_X[0, :, :], cmap = 'gray')
plt.title("Ground Truth: {}".format(test_Y[0]))

In [ ]:
train_X = train_X.reshape(-1, 28, 28, 1)
test_X = test_X.reshape(-1, 28, 28, 1)

print('Dimensions of the training set: ', train_X.shape)
print('Dimensions of the test set: ', test_X.shape)

In [ ]:
train_X = train_X.astype('float32')
test_X = test_X.astype('float32')

train_X = train_X / 255.
test_X = test_X / 255.

# Change the labels from categorical to one-hot encoding
train_Y_one_hot = to_categorical(train_Y)
test_Y_one_hot = to_categorical(test_Y)

# Display the change for category label using one-hot encoding
print('Original label:', train_Y[0])
print('After conversion to one-hot:', train_Y_one_hot[0])

In [ ]:
train_X, valid_X, train_label, valid_label = train_test_split(train_X,
                                                              train_Y_one_hot,
                                                              test_size = 0.2,
                                                              random_state = 13)

print('Dimensions of the training set (X): ', train_X.shape)
print('Dimensions of the validation set (X): ', valid_X.shape)
print('Dimensions of the test set (Y): ', train_label.shape)
print('Dimensions of the test validation set (Y): ', valid_label.shape)

### Baseline CNN Model

In [ ]:
batch_size = 5000
epochs = 10
num_classes = 10
input_shape = (28, 28, 1)

#### Building

In [ ]:
baseline_model = Sequential()
baseline_model.add(Conv2D(16, kernel_size = (3, 3), activation = 'linear', input_shape = input_shape, padding = 'same'))
baseline_model.add(LeakyReLU(alpha = 0.1))
baseline_model.add(MaxPooling2D((2, 2), padding = 'same'))
baseline_model.add(Conv2D(32, (3, 3), activation = 'linear', padding = 'same'))
baseline_model.add(LeakyReLU(alpha = 0.1))
baseline_model.add(MaxPooling2D(pool_size = (2, 2), padding = 'same'))
baseline_model.add(Conv2D(64, (3, 3), activation = 'linear', padding = 'same'))
baseline_model.add(LeakyReLU(alpha = 0.1))
baseline_model.add(MaxPooling2D(pool_size = (2, 2), padding = 'same'))
baseline_model.add(Flatten())
baseline_model.add(Dense(64, activation = 'linear'))
baseline_model.add(LeakyReLU(alpha = 0.1))      
baseline_model.add(Dense(num_classes, activation = 'softmax'))

baseline_model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])

baseline_model.summary()

#### Training

In [ ]:
baseline_model_train = baseline_model.fit(train_X,
                                 train_label,
                                 batch_size = batch_size,
                                 epochs = epochs,
                                 verbose = 1,
                                 validation_data = (valid_X, valid_label))


#### Evaluation

In [ ]:
baseline_test_eval = baseline_model.evaluate(test_X,
                                             test_Y_one_hot,
                                             verbose = 1)

print('Test loss:', baseline_test_eval[0])
print('Test accuracy:', baseline_test_eval[1])

In [ ]:
accuracy = baseline_model_train.history['accuracy']
val_accuracy = baseline_model_train.history['val_accuracy']
loss = baseline_model_train.history['loss']
val_loss = baseline_model_train.history['val_loss']

epochs = range(len(accuracy))

plt.plot(epochs, accuracy, 'bo', label = 'Training accuracy')
plt.plot(epochs, val_accuracy, 'r', label = 'Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'bo', label = 'Training loss')
plt.plot(epochs, val_loss, 'r', label = 'Validation loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

#### Prediction

In [ ]:
predicted_classes = baseline_model.predict(test_X)

predicted_classes = np.argmax(np.round(predicted_classes), axis = 1)

print('The shape of the predicted classes is:', predicted_classes.shape)
print('The shape of the test set (Y) is:', test_Y.shape)

In [ ]:
correct = np.where(predicted_classes == test_Y)[0]
print("Found %d correct labels" % len(correct))

for i, correct in enumerate(correct[:9]):
    plt.subplot(3, 3, i+1)
    plt.imshow(test_X[correct].reshape(28, 28), cmap = 'gray', interpolation = 'none')
    plt.title("Predicted {}, Class {}".format(predicted_classes[correct], test_Y[correct]))
    plt.tight_layout()

In [ ]:
incorrect = np.where(predicted_classes != test_Y)[0]
print("Found %d incorrect labels" % len(incorrect))

for i, incorrect in enumerate(incorrect[:9]):
    plt.subplot(3, 3, i+1)
    plt.imshow(test_X[incorrect].reshape(28, 28), cmap = 'gray', interpolation = 'none')
    plt.title("Predicted {}, Class {}".format(predicted_classes[incorrect], test_Y[incorrect]))
    plt.tight_layout()

plt.savefig('440_incorrect.png')

In [ ]:
import warnings
warnings.filterwarnings('ignore')

target_names = ["Class {}".format(i) for i in range(num_classes)]
print(classification_report(test_Y, predicted_classes, target_names = target_names))

### a.) Visualizing Intermediate Activations

In [ ]:
kernels = baseline_model.get_layer(name = 'conv2d').get_weights()[0][:, :, 0, :]
print(kernels)

In [ ]:
for i in range(0, 3):
  for j in range(0, 3):
    plt.plot(kernels[i][j])
    plt.show()

In [ ]:
for i in range(0, 3):
  for j in range(0, 3):
    plt.plot(kernels[i][j])

plt.show()

In [ ]:
for kernel in kernels:
  plt.plot(kernel)

plt.show()

### b.) Visualizing Intermediate Activations

In [ ]:
# choose any image to want by specifying the index
img_to_visualize = train_X[51]

# Keras requires the image to be in 4D, so we add an extra dimension to it.
img_to_visualize = np.expand_dims(img_to_visualize, axis=0)

In [ ]:
layer_outputs = [layer.output for layer in baseline_model.layers]
activation_model = Model(inputs = baseline_model.input, outputs = layer_outputs)

img = img_to_visualize.reshape(1, 28, 28, 1)
fig = plt.figure(figsize = (5, 5))
plt.imshow(img[0, :, :, 0], cmap = "gray")
plt.axis('off')

In [ ]:
activations = activation_model.predict(img)

layer_names = []
for layer in baseline_model.layers[0:3]:
    layer_names.append(layer.name) # Names of the layers, so you can have them as part of your plot
    
images_per_row = 16
for layer_name, layer_activation in zip(layer_names, activations): # Displays the feature maps
    n_features = layer_activation.shape[-1] # Number of features in the feature map
    size = layer_activation.shape[1] # The feature map has shape (1, size, size, n_features).
    n_cols = n_features // images_per_row # Tiles the activation channels in this matrix
    display_grid = np.zeros((size * n_cols, images_per_row * size))
    for col in range(n_cols): # Tiles each filter into a big horizontal grid
        for row in range(images_per_row):
            channel_image = layer_activation[0, :, :, col * images_per_row + row]
            channel_image -= channel_image.mean() # Post-processes the feature to make it visually palatable
            channel_image /= channel_image.std()
            channel_image *= 64
            channel_image += 128
            channel_image = np.clip(channel_image, 0, 255).astype('uint8')
            display_grid[col * size : (col + 1) * size, # Displays the grid
                         row * size : (row + 1) * size] = channel_image
    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1],
                        scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect = 'auto', cmap = 'viridis')

In [ ]:
def display_activation(activations, col_s, row_s, act_index):
  activation = activations[act_index]
  activation_index = 0
  fig, ax = plt.subplots(row_s, col_s, figsize = (row_s*2.5, col_s*1.5))
  for row in range(0, row_s):
    for col in range(0,col_s):
      ax[row][col].imshow(activation[0, :, :, activation_index], cmap = 'gray')
      activation_index += 1

for i in range(0, 3):
  display_activation(activations, 2, 2, i) 

### c.) cnnvis.py

# Day 7. (Tuesday 16th August):

## Exercise 5:

### 1.) Feed-Forward Neural Network on MNIST:

### 2.) Recurrent Neural Network on IMDb:

#### Data Import

In [ ]:
from keras.datasets import imdb
from keras.preprocessing import sequence

# Load the dataset but only keep the top n words, zero the rest
top_words = 10000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = top_words)

# Truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen = max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen = max_review_length)

print(X_train.shape)
print(X_test.shape)

#### Building

In [ ]:
from keras.layers import Embedding, LSTM

# Create the model
embedding_vecor_length = 128

visible = Input(shape = max_review_length)
hidden1 = Embedding(top_words, embedding_vecor_length, input_length = max_review_length)(visible)
hidden2 = LSTM(60, activation = 'tanh')(hidden1)
output = Dense(1, activation = 'sigmoid')(hidden2)
model = Model(inputs = visible, outputs = output)

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

print(model.summary())

#### Training

In [ ]:
history = model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 10, batch_size = 128)

#### Evaluation

In [ ]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(accuracy))

plt.plot(epochs, accuracy, 'bo', label = 'Training accuracy')
plt.plot(epochs, val_accuracy, 'r', label = 'Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'bo', label = 'Training loss')
plt.plot(epochs, val_loss, 'r', label = 'Validation loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()